In [0]:
#Please re-run if it crashes
import numpy
import numpy as np
from google.colab import drive

from scipy.stats import norm

import keras
from keras import layers
from keras.models import Model
from keras import metrics
from keras import backend as K   # 'generic' backend so code works with either tensorflow or theano

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [0]:
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [0]:
# dimensions of our images.
img_width, img_height = 250, 250

train_data_dir = '/content/drive/My Drive/Colab Notebooks/data/train'
validation_data_dir = '/content/drive/My Drive/Colab Notebooks/data/validation'
nb_train_samples = 1139
nb_validation_samples = 100
epochs = 5
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#input_shape = Lambda(lambda x: tf.image.rgb_to_grayscale(x))

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [0]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 150x150
        #color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')


Found 1119 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [0]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
model.save_weights('/content/drive/My Drive/Colab Notebooks/faceWeights.h5')  # always save your weights after training or during training

Epoch 1/5
71/71 [==============================] - 14s 201ms/step - loss: 0.7274 - acc: 0.5860 - val_loss: 0.6947 - val_acc: 0.5104
Epoch 2/5
71/71 [==============================] - 14s 203ms/step - loss: 0.6462 - acc: 0.6434 - val_loss: 0.5876 - val_acc: 0.6905
Epoch 3/5
71/71 [==============================] - 14s 197ms/step - loss: 0.5681 - acc: 0.7065 - val_loss: 0.6894 - val_acc: 0.5595
Epoch 4/5
71/71 [==============================] - 14s 202ms/step - loss: 0.4681 - acc: 0.7867 - val_loss: 0.4870 - val_acc: 0.7738
Epoch 5/5
71/71 [==============================] - 14s 197ms/step - loss: 0.3939 - acc: 0.8404 - val_loss: 0.4868 - val_acc: 0.7976


In [0]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy

test_data_dir = '/content/drive/My Drive/Colab Notebooks/data/test'
test_datagen = ImageDataGenerator(rescale=1./255)

#batch_input = np.zeros((1,) + (250, 250, 3), dtype=K.floatx())
#img_input = img_to_array(load_img(test_data_dir))
#batch_input[0] = img_input

#test_data_dir = test_data_dir.reshape(img_width, img_height)
test_generator = test_datagen.flow_from_directory(
        #batch_input,
        test_data_dir,
        target_size=(img_width, img_height),
        #color_mode="grayscale",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)

y_pred = numpy.rint(predict)
y_true = test_generator.classes
print (confusion_matrix(y_true, y_pred))

Found 40 images belonging to 2 classes.
[[18  2]
 [ 6 14]]
